# 🎨 MASUKA - AI Generation Platform

**Train LoRAs and generate ultra-realistic images with NO restrictions**

---

## 🚀 One-Click Setup

1. **Enable GPU**: Runtime → Change runtime type → T4 GPU (or A100 for faster training)
2. **Run the cell below**: Everything is automated!
   - Downloads latest code
   - Runs database migrations
   - Validates schema
   - Installs dependencies
   - Launches backend & frontend
3. **Access GUI**: Click the URL when it appears

---

## ✨ Features

- **LoRA Training**: Train character LoRAs with your images (5-15 min)
- **Image Generation**: Create unlimited images with trained characters
- **Dataset Management**: Upload, organize, and caption images
- **Auto-Migration**: Database schema automatically updated
- **Safety Nets**: Validates everything before launch

---

## 🔄 To Update & Restart

Just stop and re-run the cell below. It will:
- Pull latest code from GitHub
- Apply any new database migrations
- Restart everything fresh

**No manual steps needed!**

---

In [ ]:
# 🚀 MASUKA - Complete Setup, Migration & Launch
# This single cell handles everything:
# - Cleans previous installations
# - Downloads latest code
# - Runs database migrations
# - Validates schema
# - Launches backend & frontend
# - Provides access URL

import os
import sys
import time
import subprocess
from pathlib import Path

print("=" * 70)
print("🎨 MASUKA - AI Generation Platform")
print("=" * 70)

# ============================================================================
# STEP 1: CLEANUP & DOWNLOAD
# ============================================================================
print("\n📦 STEP 1/5: Cleaning and downloading latest version...")
print("-" * 70)

# Clean up previous installations
print("🧹 Removing previous installations...")
!rm -rf /content/CharForgex /content/ai-gen /content/masuka 2>/dev/null || true
!find /content -type d -name __pycache__ -exec rm -rf {} + 2>/dev/null || true
print("✓ Cleanup complete")

# Clone latest version
print("\n📥 Downloading from GitHub...")
!git clone -q https://github.com/SamuelD27/ai-gen.git /content/ai-gen 2>&1 | grep -v "Cloning into" || true
os.chdir('/content/ai-gen')
!git pull -q origin main 2>&1 | grep -v "Already up to date" || true
print("✓ Latest code downloaded")

# ============================================================================
# STEP 2: DATABASE MIGRATION & VALIDATION
# ============================================================================
print("\n🗄️  STEP 2/5: Database migration & validation...")
print("-" * 70)

os.chdir('/content/ai-gen/charforge-gui/backend')

# Ensure migrations directory exists
migrations_dir = Path('migrations')
if not migrations_dir.exists():
    print("⚠️  Warning: migrations directory not found")
else:
    print(f"✓ Found {len(list(migrations_dir.glob('*.py')))} migration files")

# Run migrations
print("\n🔄 Running database migrations...")
try:
    result = !python run_migrations.py 2>&1
    for line in result:
        if '✓' in line or 'Added' in line or 'already exists' in line or 'Completed' in line:
            print(f"  {line}")
    print("✓ Database migrations complete")
except Exception as e:
    print(f"⚠️  Migration warning: {e}")
    print("  Attempting to create tables manually...")
    !python -c "from app.core.database import Base, engine; Base.metadata.create_all(engine)" 2>&1

# Validate database schema
print("\n🔍 Validating database schema...")
try:
    validation = !python -c "
import sys
sys.path.insert(0, '.')
from app.core.database import engine
from sqlalchemy import text, inspect

conn = engine.connect()
inspector = inspect(engine)

# Check if training_sessions table exists
tables = inspector.get_table_names()
if 'training_sessions' not in tables:
    print('❌ training_sessions table not found')
    sys.exit(1)

# Check if error_message column exists
columns = [col['name'] for col in inspector.get_columns('training_sessions')]
if 'error_message' in columns:
    print('✓ error_message column exists')
else:
    print('❌ error_message column missing')
    # Try to add it
    try:
        conn.execute(text('ALTER TABLE training_sessions ADD COLUMN error_message TEXT'))
        conn.commit()
        print('✓ Added error_message column')
    except Exception as e:
        print(f'⚠️  Could not add column: {e}')

# Verify all required tables
required_tables = ['users', 'characters', 'training_sessions', 'datasets', 'dataset_images']
missing = [t for t in required_tables if t not in tables]
if missing:
    print(f'⚠️  Missing tables: {missing}')
else:
    print(f'✓ All {len(required_tables)} required tables present')

conn.close()
" 2>&1
    
    for line in validation:
        print(f"  {line}")
        
except Exception as e:
    print(f"⚠️  Validation error: {e}")
    print("  Continuing anyway...")

# ============================================================================
# STEP 3: INSTALL DEPENDENCIES
# ============================================================================
print("\n📚 STEP 3/5: Installing dependencies...")
print("-" * 70)

os.chdir('/content/ai-gen')
print("⚙️  Running setup script (this takes 10-15 minutes on first run)...")
print("  ⏳ Installing packages, downloading models...")

# Run setup with output filtering
!python masuka_colab_setup.py 2>&1 | grep -E "(✓|✗|🚀|🌐|http|Error|Failed|Success)" || python masuka_colab_setup.py

# ============================================================================
# STEP 4: HEALTH CHECK
# ============================================================================
print("\n💚 STEP 4/5: System health check...")
print("-" * 70)

# Wait for backend to start
print("⏳ Waiting for backend to initialize...")
time.sleep(10)

# Check if backend is running
try:
    import requests
    response = requests.get("http://localhost:8000/health", timeout=5)
    if response.status_code == 200:
        print("✓ Backend is healthy and responding")
    else:
        print(f"⚠️  Backend returned status {response.status_code}")
except Exception as e:
    print(f"⚠️  Backend health check failed: {e}")
    print("  Backend might still be starting...")

# Check database connection
try:
    check_db = !python -c "
import sys
sys.path.insert(0, 'charforge-gui/backend')
from app.core.database import SessionLocal
try:
    db = SessionLocal()
    db.execute('SELECT 1')
    db.close()
    print('✓ Database connection successful')
except Exception as e:
    print(f'❌ Database error: {e}')
    sys.exit(1)
" 2>&1
    for line in check_db:
        print(f"  {line}")
except:
    pass

# ============================================================================
# STEP 5: LAUNCH COMPLETE
# ============================================================================
print("\n" + "=" * 70)
print("✨ MASUKA IS READY!")
print("=" * 70)

# Find and display the ngrok URL
print("\n🌐 Access your GUI at:")
time.sleep(2)

# Try to get ngrok URL from logs
try:
    ngrok_check = !curl -s http://localhost:4040/api/tunnels 2>/dev/null || echo ""
    if ngrok_check and len(ngrok_check) > 0 and 'public_url' in str(ngrok_check):
        import json
        try:
            tunnels = json.loads(' '.join(ngrok_check))
            if 'tunnels' in tunnels and len(tunnels['tunnels']) > 0:
                url = tunnels['tunnels'][0]['public_url']
                print(f"\n  🔗 {url}")
                print(f"\n  Copy this URL and paste it in your browser!")
        except:
            pass
except:
    pass

print("\n📊 System Status:")
print("  ✓ Code: Latest version from GitHub")
print("  ✓ Database: Migrated and validated")
print("  ✓ Backend: Running on port 8000")
print("  ✓ Frontend: Running with ngrok tunnel")

print("\n💡 Quick Tips:")
print("  • Upload images in the Media tab")
print("  • Create datasets with trigger words")
print("  • Train LoRAs (takes 5-15 minutes)")
print("  • Generate unlimited images!")

print("\n🔄 To restart with latest code:")
print("  1. Stop this cell (⏹️ button)")
print("  2. Re-run this cell")
print("     Database migrations will run automatically")

print("\n" + "=" * 70)
print("🎉 Ready to create! Have fun!")
print("=" * 70)

In [ ]:
# CELL 3: Run Database Migrations
# Run this cell after pulling new code to update the database schema
# This is safe to run multiple times - migrations will skip if already applied

print("🔄 Running database migrations...\n")

import os
import sys

# Navigate to backend directory
os.chdir('/content/ai-gen/charforge-gui/backend')

# Run migrations
print("=" * 60)
!python run_migrations.py
print("=" * 60)

print("\n✓ Database migrations complete!")
print("\n💡 If you see 'column already exists' messages, that's normal.")
print("   It means your database is already up to date.")
print("\n🚀 Your database is now ready to use!")